In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import feather
from datetime import datetime
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
df = feather.read_dataframe('feature/pred_feature_100_200.feather')
df_id = df[['user_id', 'item_id']]
df_X = df.drop(columns=['user_id', 'item_id'])
if 'index_x' in df.columns:
    df_X = df_X.drop(columns=['index_x', 'index_y'])

# 数据预处理
scaler = StandardScaler()
df_X = scaler.fit_transform(df_X)

X = torch.tensor(df_X, dtype=torch.float32)

In [5]:
# 定义神经网络模型
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()   
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x
        
model = NeuralNetwork(X.shape[1]).to(device)
model.load_state_dict(torch.load('model/model_epoch10.pth'))

<All keys matched successfully>

In [ ]:
X = X.to(device)
# 输出所有正样本的id
model.eval()
with torch.no_grad():
    y_pred = model(X)
    y_pred = y_pred.cpu().numpy().flatten()
    df_id['score'] = y_pred
    df_id = df_id[df_id['score'] > 0.5]
    df_id.drop(columns=['score'], inplace=True)
    df_id.to_csv('result/pred_result_{:}.txt'.format(datetime.now().strftime("%m%d%H%M")), index=False, header=False, sep='\t')
    
    

c:\DevTools\anaconda3\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [4]:
total_num = X.shape[0]
times = 10
batch_size = total_num // times

# 输出所有正样本的id
model.eval()

with torch.no_grad():
    y_pred_final = []
    for i in range(times):
        X_batch = X[i*batch_size:(i+1)*batch_size]
        X = X_batch.to(device)
        y_pred = model(X)
        y_pred = y_pred.cpu().numpy().flatten()
        y_pred_final.extend(y_pred)
    df_id['score'] = y_pred
    df_id = df_id[df_id['score'] > 0.5]
    df_id.drop(columns=['score'], inplace=True)
    df_id.to_csv('result/pred_result_{:}.txt'.format(datetime.now().strftime("%m%d %H%M")), index=False, header=False, sep='\t')
    
    

ValueError: Length of values (0) does not match length of index (7231448)

In [8]:
df_id.to_csv('result/pred_result_{:}.txt'.format(datetime.now().strftime("%m%d %H%M")), index=False, header=False, sep='\t')